In [5]:
!pip install pylibjpeg-libjpeg
!pip install pylibjpeg
!pip install pydicom
!pip install keras-cv-attention-models
!pip install dicomsdl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 66.7 MB/s eta 0:00:00


In [2]:
import os
import pydicom
import cv2
import numpy as np
from tensorflow.keras import models, layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [3]:
# Define directory paths
cancer_dir = "train/cancer"
no_cancer_dir = "train/noCancer"

# Image dimensions
img_width, img_height = 256, 256

# Function to load and preprocess images
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = pydicom.dcmread(os.path.join(folder, filename)).pixel_array
        img = cv2.resize(img, (img_width, img_height))
        if img is not None:
            images.append(img)
            labels.append(label)
    return images, labels

In [4]:
# Load images
cancer_images, cancer_labels = load_images_from_folder(cancer_dir, 1)
no_cancer_images, no_cancer_labels = load_images_from_folder(no_cancer_dir, 0)

In [5]:
# Combine and normalize images
images = np.array(cancer_images + no_cancer_images) / 255.0
labels = np.array(cancer_labels + no_cancer_labels)

In [6]:
# One-hot encode labels
labels = to_categorical(labels, num_classes=2)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Reshape data to fit model
X_train = X_train.reshape(-1, img_width, img_height, 1)
X_test = X_test.reshape(-1, img_width, img_height, 1)

In [ ]:
# Build model
def build_cnn(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

# Define model parameters
input_shape = (img_width, img_height, 1)
num_classes = 2

In [ ]:
# Build and compile model
model = build_cnn(input_shape, num_classes)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

In [ ]:
# Evaluate the modelo
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
# accuracy train & valid
plt.figure(figsize=[6,4])
plt.plot(history.history['accuracy'], 'black', linewidth=2.0)
plt.plot(history.history['val_accuracy'], 'blue', linewidth=2.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'], fontsize=14)
plt.xlabel('Epochs', fontsize=10)
plt.ylabel('Accuracy', fontsize=10)
plt.title('Accuracy Curves', fontsize=12)
plt.show()

In [ ]:
# loss train & valid
plt.figure(figsize=[6,4])
plt.plot(history.history['loss'], 'black', linewidth=2.0)
plt.plot(history.history['val_loss'], 'blue', linewidth=2.0)
plt.legend(['Training Loss', 'Validation Loss'], fontsize=14)
plt.xlabel('Epochs', fontsize=10)
plt.ylabel('Loss', fontsize=10)
plt.title('Loss Curves', fontsize=12)
plt.show()